# One Hot Vector化サンプル

In [1]:
# 分析対象CSVのダウンロード
!wget https://github.com/MattyMc/is-a-tent/raw/master/GoSales_Tx_LogisticRegression.csv

--2018-05-13 11:02:57--  https://github.com/MattyMc/is-a-tent/raw/master/GoSales_Tx_LogisticRegression.csv
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MattyMc/is-a-tent/master/GoSales_Tx_LogisticRegression.csv [following]
--2018-05-13 11:02:57--  https://raw.githubusercontent.com/MattyMc/is-a-tent/master/GoSales_Tx_LogisticRegression.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1616445 (1.5M) [text/plain]
Saving to: ‘GoSales_Tx_LogisticRegression.csv.1’

100%[======================================>] 1,616,445   --.-K/s   in 0.04s   

2018-05-13 11:02:58 (41.5 MB/s) - ‘GoSales_Tx_LogisticRegression.csv.1’ 

In [2]:
# 分析対象データをDataFrameにロード
import pandas as pd
import numpy as np

df = pd.read_csv('GoSales_Tx_LogisticRegression.csv', header=0)
df.head()

,IS_TENT,GENDER,AGE,MARITAL_STATUS,PROFESSION
0,False,M,27,Single,Professional
1,False,F,39,Married,Other
2,False,F,39,Married,Other
3,False,F,56,Unspecified,Hospitality
4,False,M,45,Married,Retired


In [3]:
# One Hot化
# 入力データ項目のうち、GENDER, MARITAL_STATUS, PROFESSIONをOne Hot化します
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [4]:
profession = df['PROFESSION'].values
profession_enc = preprocessing.LabelEncoder().fit_transform(profession).reshape(-1,1)
profession_enc2 = OneHotEncoder().fit_transform(profession_enc).toarray()

In [5]:
# 結果の確認をします
print(profession_enc[:5])
print(profession_enc2[:5])

[[3]
 [2]
 [2]
 [1]
 [5]]
[[ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.]]


In [6]:
# 他の2項目についても同じ処理を行います
marital_status = df['MARITAL_STATUS'].values
marital_status_enc = preprocessing.LabelEncoder().fit_transform(marital_status).reshape(-1,1)
marital_status_enc2 = OneHotEncoder().fit_transform(marital_status_enc).toarray()

gender = df['GENDER'].values
gender_enc = preprocessing.LabelEncoder().fit_transform(gender).reshape(-1,1)
gender_enc2 = OneHotEncoder().fit_transform(gender_enc).toarray()

In [7]:
# AGEに関しては、浮動小数点に変換後、標準化します
from sklearn.preprocessing import MinMaxScaler
age = df['AGE'].values.reshape(-1, 1).astype(np.float32)
age_std = MinMaxScaler().fit_transform(age)

# 結果確認
print(age_std[:5])

[[ 0.19230768]
 [ 0.4230769 ]
 [ 0.4230769 ]
 [ 0.75      ]
 [ 0.53846157]]


In [8]:
# 今までの変換結果を元に学習用配列を作成します
x = np.hstack([gender_enc2, age_std, marital_status_enc2, profession_enc2])
# 結果確認
pd.DataFrame(x).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,1.0,0.192308,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.423077,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.423077,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.750000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.538462,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [12]:
# 教師データの作成
is_tent = df['IS_TENT'].values
y = preprocessing.LabelEncoder().fit_transform(is_tent).ravel()

# 結果確認
print(y[:5])
y.shape

[0 0 0 0 0]


(60252,)

In [13]:
# 以下はおまけです
from sklearn import svm
from sklearn.model_selection import train_test_split

# 訓練データと検証データ取得
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# モデル作成
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1.0)

# 学習
lr.fit(x_train, y_train)

# 検証データで予測
y_pred = lr.predict(x_test)
lr.score(x_test, y_test)

0.89743589743589747